<a href="https://colab.research.google.com/github/softmurata/promptparty/blob/main/evaluation/opencliptorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation

In [2]:
!pip install -q open-clip-torch transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.0 MB/s eta 0:00:00


In [12]:
!pip install -q sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Get Demo data

In [ ]:
# Get source data
!wget https://img.freepik.com/free-photo/picture-frame-on-a-wall-with-scandinavian-home-interior_53876-139779.jpg -O /content/room001.jpg

# Get target data
!wget https://img.freepik.com/free-psd/modern-interior-design-living-room_176382-1266.jpg -O /content/room002.jpg
!wget https://img.freepik.com/premium-photo/living-room-with-white-couch-coffee-table-with-white-rug-floor_784625-7082.jpg -O /content/room003.jpg
!wget https://img.freepik.com/free-photo/modern-living-room-interior-design_23-2150794674.jpg -O /content/room004.jpg
!wget https://img.freepik.com/premium-photo/living-room-with-large-plant-white-pot_784625-7578.jpg -O /content/room005.jpg

CoCA

In [4]:
import open_clip
import torch
from PIL import Image

from sentence_transformers import SentenceTransformer
from scipy.spatial import distance


en_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

model, _, transform = open_clip.create_model_and_transforms(
  model_name="coca_ViT-L-14",
  pretrained="mscoco_finetuned_laion2B-s13B-b90k"
)

In [10]:
def get_clip_features(img_path):

  im = Image.open(img_path).convert("RGB")
  im = transform(im).unsqueeze(0)

  with torch.no_grad(), torch.cuda.amp.autocast():
      image_features = model.encode_image(im)
      generated = model.generate(im)

  text = open_clip.decode(generated[0]).split("<end_of_text>")[0].replace("<start_of_text>", "")

  return image_features.detach().cpu().numpy(), text

In [20]:
source_img_fea, source_gen_text = get_clip_features("room001.jpg")
target_img_fea, target_gen_text = get_clip_features("room003.jpg")

print(source_gen_text, target_gen_text)

source_sentences = [source_gen_text]
target_sentences = [target_gen_text]

#Sentences are encoded by calling model.encode()
source_embeddings = en_model.encode(source_sentences)
target_embeddings = en_model.encode(target_sentences)

tv = 1 - distance.cosine(source_embeddings.reshape(-1), target_embeddings.reshape(-1))
sv = 1 - distance.cosine(source_img_fea.reshape(-1), target_img_fea.reshape(-1))
print("text similarity score: ", tv, "image similarity score: ", sv)

a living room scene with a couch , coffee table and a painting .  a living room with a couch , coffee table and a lamp . 
text similarity score:  0.791092574596405 image similarity score:  0.8731975555419922
